In [1]:
import numpy as np
import scipy.io
import scipy
import numpy.matlib
import scipy.stats
from sklearn.decomposition import PCA

In [2]:
pca = PCA(n_components = 2) 
x = np.array([[2, 3, 3], [1, 2, 3]])
y = pca.fit_transform(x)
print y

[[ 0.70710678  0.        ]
 [-0.70710678  0.        ]]


In [3]:
def add_dependencies():
   pass 

In [4]:
add_dependencies()

In [5]:
def Config(source=None, target=None):
    amazon = 0
    webcam = 1
    dslr = 2
    caltech = 3
    
    param = {}
    param['domains'] = [amazon, webcam, dslr, caltech]
    param['domain_names'] = ['amazon', 'webcam', 'dslr', 'caltech']
    param['use_Gaussian_kernel'] = False
    
    param['categories'] = ['back_pack', 'bike', 'calculator',
                           'headphones', 'keyboard', 'laptop_computer', 'monitor', 'mouse',
                           'mug', 'projector']
    
    param['DATA_DIR'] = './MaxMarginDomainTransforms/ToRelease_GFK/data/'
    param['held_out_categories'] = False
    if source is None:
        param['source'] = webcam
    else:
        param['source'] = source
    
    if target is None:
        param['target'] = dslr
    else:
        param['target'] = target
    
    param['num_trials'] = 20
    param['dim'] = 20
    param['norm_type'] = 'l2_zscore'
    param['C_s'] = 0.05
    param['C_t'] = 1
    param['mmdt_iter'] = 2
    
    if param['source'] == amazon:
        param['num_train_source'] = 20
    else:
        param['num_train_source'] = 8
    
    param['num_train_target'] = 3
    
    param['result_filename'] = './MaxMarginDomainTransforms/DataSplitsOfficeCaltech/SameCategory_{0}-{1}_{2}RandomTrials_10Categories.mat'.format(param['domain_names'][param['source']],
                                                                     param['domain_names'][param['target']],
                                                                     param['num_trials'])
    
    return param

In [6]:
def NormData(fts, norm_type='l2_zscore'):
    sqr_fts = np.sqrt(np.sum(fts**2, 1))
    sqr_fts = np.matrix(sqr_fts).T
    sqr_fts = np.matlib.repmat(sqr_fts, 1, fts.shape[1])
    fts = fts / sqr_fts
#     switch norm_type
#         case 'l1_zscore'
#             fts = fts ./ repmat(sum(abs(fts),2),1,size(fts,2));
#             fts = zscore(fts,1);
#         case 'l2_zscore'
#             fts = fts ./ repmat(sqrt(sum(fts.^2,2)),1,size(fts,2));
#             fts = zscore(fts,1);
#         case 'l1'
#             fts = fts ./ repmat(sum(abs(fts),2),1,size(fts,2));
#         case 'l2'
#             fts = fts ./ repmat(sqrt(sum(fts.^2,2)),1,size(fts,2));
#         case 'none'
#             return;
#         otherwise
#             error('norm');
#     end
    
    return scipy.stats.zscore(fts)

In [7]:
def LoadOfficePlusCaltechData(foldername, norm_type):
    domain_names = ['amazon_SURF_L10.mat', 'webcam_SURF_L10.mat', 'dslr_SURF_L10.mat', 'Caltech10_SURF_L10.mat']
#     Data = np.zeros((len(domain_names),len(domain_names)))
#     Labels = np.zeros((len(domain_names),len(domain_names)))
    Data = []
    Labels = []
    for idx, name in enumerate(domain_names):
        fullfilename = foldername + name
        obj = scipy.io.loadmat(fullfilename)
        
        fts = obj['fts']
        labels = obj['labels']
        fts = NormData(fts, norm_type)
        Data.append(fts)
        Labels.append(labels)
        
    return Data, Labels

In [8]:
param = Config()
# param

In [9]:
[Data, Labels] = LoadOfficePlusCaltechData( param['DATA_DIR'], param['norm_type'])

In [10]:
source_domain = param['source']
target_domain = param['target']

splits = scipy.io.loadmat(param['result_filename'])
train_ids = splits['train']
test_ids = splits['test']
# Data[source_domain]
train_ids_source = train_ids[0][0][0][0]
train_ids_target = train_ids[0][0][1][0]
test_ids_source = test_ids[0][0][0][0]
test_ids_target = test_ids[0][0][1][0]

In [11]:
n = param['num_trials']
telapsed = np.zeros((n,1))
accuracy = np.zeros((n,1))
pred_labels = np.zeros((n, 1))

In [12]:
def UpdateLabels(labels, all_categories, categories):
    for i in range(0, labels.size):
        oldL = labels[i]
        l = all_categories[oldL]
#         newL = find(strcmp(l, categories));
        newL = l.index(categories)
        labels[i] = newL;
    return labels    

In [34]:
def UpdateLabelValues(labels, param):
    if 'all_categories' not in param:
        return labels
    all_categories = param['all_categories']
    categories = param['categories']
    labels['train']['source'] = UpdateLabels(labels['train']['source'], all_categories, categories)
    labels['train']['target'] = UpdateLabels(labels['train']['target'], all_categories, categories)
    labels['test']['target'] = UpdateLabels(labels['test']['target'], all_categories, categories)
    return labels

In [78]:
def TrainMmdtOneIter(labels, data, param, idx):
    data['transformed_target'] =  AugmentWithOnes(data['target'])*param['A']
    data_svm = np.vstack((AugmentWithOnes(data['source']), data['transformed_target']))

    labels_svm = np.hstack((labels['source'].T, labels['target'].T))
    
    weights_s = param['C_s']* np.ones((labels['source'].size, 1))
    weights_t = param['C_t']* np.ones((labels['target'].size, 1))
    param['weights'] = np.vstack((weights_s, weights_t))
    
    if idx == 0 and ('source_svm' in param):
        model = param['source_svm'];
    else:
        model = train(param['weights'], labels_svm.T, sps.csr_matrix(data_svm, dtype = np.float).eliminate_zeros(), '-c 1 -q')
    tstart = time.time()
    L = learnAsymmTransformWithSVM(model['w'][param['train_classes'] - 1,:], \
    param['train_classes'], AugmentWithOnes(data['target']), labels['target'], param)
    param['A'] = L.T
    param['telapsed']['idx'] = time.time() - tstart;
    return [model, data, param]

In [79]:
def AsymmetricFrob_slack_kernel2(KA,KB,C,gamma=None,thresh=None):
    #Frobenius-based transformation learning
    if thresh is None:
        thresh=10e-3;

    if gamma is None:
        gamma = 1e1;

    maxit = 1e6;
    
    [nA,nA] = KA.shape
    [nB,nB] = KB.shape
    S = np.zeros((nA,nB))
    [c,t] = C.shape
    slack = np.zeros((c,1));
    lambda1 = np.zeros((c,1));
    lambda2 = np.zeros((c,1));
    viol = -1*C[:,3]*C[:,2];
    viol = viol.T;

    for i in range(maxit):
        [mx,curri] = np.max(viol);
        if i%1000 == 0:
            print 'Iteration {}, maxviol {}'.format(i, mx)
        
        if mx < thresh:
            break;
    
        p1 = C[curri,0];
        p2 = C[curri,1];
        b = C[curri,2];
        s = C[curri,3];
        kx = KA[p1-1,:];
        ky = KB[:,p2-1];

        alpha = min(lambda1[curri],(np.matmul(s,(b-np.matmul(kx, np.matmul(S,ky))-slack[curri]))) / \
                    (1/gamma + np.matmul(KA[p1,p1],KB[p2,p2])) );
        lambda1[curri] = lambda1[curri] - alpha;
        S[p1,p2] = S[p1,p2] + s*alpha;
        slack[curri] = slack[curri] - alpha/gamma;
        alpha2 =  min(lambda2[curri],gamma*slack[curri]);
        slack[curri] = slack[curri] - alpha2/gamma;
        lambda2[curri] = lambda2[curri] - alpha2;

#         update viols
#         v = KA[C[:,1],p1];
#         w = KB[p2,C[:,2].T;
        viol = viol + np.matmul(s,  alpha *C[:,3].T * KA[C[:,0],p1].T * KB[p2,C[:,1]]);
        viol[curri] = viol[curri] + [alpha+alpha2]/gamma;
    return [S, slack, i]


In [80]:
def learnAsymmTransformWithSVM(XA, yA, XB, yB, params):
    dA = XA.shape[1];
    dB = XB.shape[1]

    C = GetConstraints(yA,yB);

    if dA != dB:
        K0aa = formKernel(XA, XA, params);
        K0bb = formKernel(XB, XB, params);

        C[:,1] = C[:,1] - len(yA);
        S = AsymmetricFrob_slack_kernel2(K0aa,K0bb,C,params['gamma'],1e-3);
        params['S'] = S;

        L = np.eye(dA, dB) + np.matmul(XA.T * np.matmul(S, XB))
    else:
        X = np.vstack((XA, XB))
        K0train = formKernel(X, X, params);
        S = asymmetricFrob_slack_kernel(K0train,C,params.gamma,10e-3);

        L = np.eye(dA) + np.matmul(X.T * np.matmul(S, X));
    return L

In [81]:
def AugmentWithOnes(data):
    return np.hstack((data, np.ones(((data.shape[0], 1)))))

def GetConstraints(y1, y2):
    pos=1;
    ly1=len(y1);
    ly2=len(y2);
    C=np.zeros((ly1*ly2,4));
    for i in range(ly1):
        for j in range(ly2):
            if(y1[i]==y2[j]):
                C[pos,:]=[i, j+ly1, 1, -1]; # w'Ax > 1; -w'Ax < -1;
            else:
                C[pos,:]=[i, j+ly1, -1, 1]; # w'Ax < -1
            pos=pos+1
    return C

In [82]:
def TrainMmdt(labels, data, param):
    if ('C_s' not in param or 'C_t' not in param):
        param['C_s'] = 1
        param['C_t'] = 1
    
    if 'gamma' not in param:
        param['gamma'] = 1e-4
    
    dA = data['source'].shape[1]
    dB = data['target'].shape[1]
    
    param['A'] = np.eye(dB + 1, dA + 1)
    
    if 'train_classes' not in param:
        param['train_classes'] = np.sort(np.unique(labels['source']))
    
    for idx in range(param['mmdt_iter']):
        [model, data, param] = TrainMmdtOneIter(labels, data, param, idx)
    
    model['w'] = model['w'] * param['A'].T
    A = param['A']
    return [model, A]

In [83]:
for i in range(0, n):
    data = {}
    data['train'] = {}
    data['test'] = {}
    print i
    data['train']['source'] = Data[source_domain][train_ids_source[i]-1][0]
    data['train']['target'] = Data[target_domain][train_ids_target[i]-1][0]
    data['test']['target'] = Data[target_domain][test_ids_target[i]-1][0]
    
    labels = {}
    labels['train'] = {}
    labels['test'] = {}
    
    labels['train']['source'] = Labels[source_domain][train_ids_source[i]-1][0]
    labels['train']['target'] = Labels[target_domain][train_ids_target[i]-1][0]
    labels['test']['target'] = Labels[target_domain][test_ids_target[i]-1][0]
#     print labels['train']
    labels = UpdateLabelValues(labels, param);
    
    if param['dim'] < np.shape(data['train']['source'])[1]:
        arr = np.array(data['train']['source'])
        arr = np.vstack((arr, data['train']['target'])) 
        arr = np.vstack((arr, data['test']['target']))
        
        pca = PCA(n_components = param['dim'])
        P = pca.fit_transform(arr.T)
#         P = princomp(arr)
        data['train']['source'] = np.matmul(data['train']['source'], P)
        data['train']['target'] = np.matmul(data['train']['target'], P)
        data['test']['target'] = np.matmul(data['test']['target'], P)
    [model_mmdt, W] = TrainMmdt(labels['train'], data['train'], param)
#     telapsed(i) = toc(tstart);
    [pl, acc, pe] = predict(labels['test']['target'].T, \
        [sparse(data['test']['target']), np.ones((np.size(labels['test']['target']),1))], \
        model_mmdt);
    accuracy[i] = acc[0];
    pred_labels[i] = pl;
    print 'Accuracy = {} (Time = {})\n'.format(accuracy[i], telapsed[i]);
print 'Mean Accuracy = {} +/- {}  (Mean time = {})'.format(mean(accuracy), std(accuracy)/sqrt(n), mean(telapsed))

0
(800, 20)
(80, 1) (30, 1)


NameError: global name 'train' is not defined